In [2]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm
import pickle
import numpy as np
import pathlib
import scipy.interpolate as ip

import  mceq_underground_helpers_oneregion_threshold as helper

In [3]:
cos_thetas = np.arange(0.5, 1.001, 0.1)
thetas = np.degrees(np.arccos(cos_thetas))

depths = [1.5,3.5]

In [4]:
file = "/hetghome/khymon/cs-files/surface_fluxes_parent_apr_GSF_Sibyll23c.pkl"


with open(file, "rb") as f:
            data = pickle.load(f)

_, cos_thetas, *fluxes = data
surface_fluxes  = {"tot": np.asarray(fluxes[0]), "conv": np.asarray(fluxes[1]), "pr": np.asarray(fluxes[2]), "k": np.asarray(fluxes[3]),
                   "pi": np.asarray(fluxes[4])}


    

In [5]:
surface_fluxes["pr"].shape

(6, 121)

In [6]:
tensor_fname = (
   "water_0.997_1000000_Survival_Probabilities.txt"
)

In [7]:
helper.load_survival_probability_tensor(file_name=tensor_fname)

utensor = helper.load_survival_probability_tensor().swapaxes(0, 1).swapaxes(1, 2)

In [8]:
for key in surface_fluxes:
        print(f"Key: {key}")  # Print the key
        print(f"Length of cos_thetas: {len(cos_thetas)}") 

Key: tot
Length of cos_thetas: 6
Key: conv
Length of cos_thetas: 6
Key: pr
Length of cos_thetas: 6
Key: k
Length of cos_thetas: 6
Key: pi
Length of cos_thetas: 6


In [9]:
intp_surface_fluxes = {
    key: ip.interp1d(cos_thetas, flux, axis=0, kind="linear")
    for key, flux in surface_fluxes.items()
}

In [10]:
# calculate underground flux fpr specific flux type and angle
def uflux(depth,key,angle):
    cth = np.cos(np.radians(angle))

    # Directly set the maximum value to 1.0 for cos(theta)
    cos_thetas_max_adjusted = 1.0  # Explicitly set the upper bound to 1.0

    # Check that the min and max of cth are within the bounds of cos_thetas
    assert np.min(cth) >= cos_thetas[0] and np.max(cth) <= cos_thetas_max_adjusted

    if np.isscalar(depth):
    
        
        # depth = np.array([depth])
        fl = intp_surface_fluxes[key](cth)[:helper.dim_ug]
        idx = np.argmax(helper.slant_depths > depth)
        frange = (
            utensor[idx - 1 : idx + 1].dot(np.nan_to_num(fl * helper._e_widths)) / helper._e_widths
        )
        with np.errstate(all="ignore"):
            return np.nan_to_num(
                np.exp(
                    ip.interp1d(
                        helper.slant_depths[idx - 1 : idx + 1],
                        np.log(frange),
                        axis=0,
                        kind="linear",
                    )(depth)
                )
            )


In [11]:
uflux(1.5,"tot",60.).shape

(91,)

In [12]:
ufluxes_tot = np.ndarray(shape=(len(thetas),len(depths),91))
ufluxes_conv = np.ndarray(shape=(len(thetas),len(depths),91))
ufluxes_pr = np.ndarray(shape=(len(thetas),len(depths),91))
ufluxes_pi = np.ndarray(shape=(len(thetas),len(depths),91))
ufluxes_k = np.ndarray(shape=(len(thetas),len(depths),91))


for t in range(len(cos_thetas)):
    for d in range(len(depths)):
        ufluxes_tot[t][d] = uflux(depths[d],"tot",thetas[t])
        ufluxes_conv[t][d] = uflux(depths[d],"conv",thetas[t])
        ufluxes_pr[t][d] = uflux(depths[d],"pr",thetas[t])
        ufluxes_pi[t][d] = uflux(depths[d],"pi",thetas[t])
        ufluxes_k[t][d] = uflux(depths[d],"k",thetas[t])

In [13]:
# avg flux accounting for solid angle
#solid_angles = 2 * np.pi * (cos_thetas[:-1] - cos_thetas[1:])  # Shape: (N,)

# Weighted average over zenith
#avg_flux = np.sum(ufluxes_tot* solid_angles) / np.sum(solid_angles)

In [28]:
plot_dir = '/hetghome/khymon/Plots/cs-analysis/'

for t in range(len(cos_thetas)):
    for d in range(len(depths)):

        A4_width_inches = 0.5*8.27  # Width of A4 paper in inches
        A4_height_inches = A4_width_inches * 0.75 # Height is one-third of width
        figsize = (A4_width_inches, A4_height_inches)
        # Create subplots using GridSpec
        fig = plt.figure(figsize=figsize, dpi=500)
        fig.tight_layout(pad=0.4)
        plt.rcParams.update({'font.size': 11})
        #plt.style.use('science')
        plt.style.use('tableau-colorblind10')
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
        # Create a gridspec with 2 rows and 1 column, where the second row will be narrow
        gs = fig.add_gridspec(2, 1, height_ratios=[3, 1], hspace=.1)  # 3:1 ratio between upper and lower plots
        # Upper plot for the spectrum
        ax1 = fig.add_subplot(gs[0])
        # Lower plot for the fraction
        ax2 = fig.add_subplot(gs[1], sharex=ax1)
        # Plot lines with labels (same as before)
        ax1.plot(helper.mute_energies, helper.mute_energies**2.7*ufluxes_tot[t][d], color='black', label='total')
        ax1.plot(helper.mute_energies,  helper.mute_energies**2.7*ufluxes_pi[t][d], color=colors[0], label=r'$\pi$')
        ax1.plot(helper.mute_energies,  helper.mute_energies**2.7*ufluxes_k[t][d], color=colors[1], label=r'K')
        ax1.plot(helper.mute_energies,  helper.mute_energies**2.7*ufluxes_pr[t][d], color=colors[2], label=r'prompt')

        # Axes labels and text (same as before)
        #ax1.set_xlabel('$E$/GeV')
        ax1.set_ylabel(r'$E^{2.7}$ $\Phi_{\mu}$ /' + '\n' + r'GeV$^{1.7}$ cm$^{-2}$s$^{-1}$sr$^{-1}$')

        ax1.text(0.35, 0.05, str(np.round(thetas[t],decimals=2))+ '$^{\circ}$', verticalalignment='bottom', horizontalalignment='right', transform=ax1.transAxes, color=colors[3])
        plt.setp(ax1.get_xticklabels(), visible=False)
        # Log scales and limits (same as before)
        ax1.set_xscale('log')
        ax1.set_yscale('log')
        ax1.set_xlim(100., 1e6)
        ax1.set_ylim(1.e-6, 1.e-1)
        ax1.minorticks_on()
        ax1.tick_params(axis='x', which='both', direction='in', bottom=True, top=True)
        ax1.tick_params(axis='y', which='both', direction='in', left=True, right=True)
        # Dummy lines for January and July to show in legend in color[3]
        ax1.set_title('$d$=' +str(depths[d])+'km')
        ax1.legend( loc='upper right', frameon=False,ncol=2)
        # Legend for colors, placed above the plot
        handles, labels = ax1.get_legend_handles_labels()
        #fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.55, 1.15), ncol=2)
        # Plot fraction of each parent to the total flux in the lower plot

        ax2.plot(helper.mute_energies, ufluxes_pi[t][d]/ufluxes_tot[t][d], label=r'Fraction from $K$', color=colors[0])
        ax2.plot(helper.mute_energies, ufluxes_k[t][d]/ufluxes_tot[t][d], label=r'Fraction from $\pi$', color=colors[1])
        ax2.plot(helper.mute_energies, ufluxes_pr[t][d]/ufluxes_tot[t][d], label=r'Fraction from prompt', color=colors[2])

        ax2.set_xlabel('$E$/GeV')
        ax2.set_ylabel('Fraction ')
        # Set limits for the lower plot
        ax2.set_xscale('log')
        ax2.set_ylim(-0.05, 1.05)
        ax2.set_xlim(100., 1e6)

        fig.savefig(plot_dir + 'uspectrum_parent_d' + str(depths[d]) + '_costheta' + str(cos_thetas[t]) + '.pdf',dpi=500,bbox_inches='tight')
        fig.clf()

<>:32: SyntaxWarning: invalid escape sequence '\c'
<>:32: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_4025577/2962969493.py:32: SyntaxWarning: invalid escape sequence '\c'
  ax1.text(0.35, 0.05, str(np.round(thetas[t],decimals=2))+ '$^{\circ}$', verticalalignment='bottom', horizontalalignment='right', transform=ax1.transAxes, color=colors[3])
/tmp/ipykernel_4025577/2962969493.py:50: RuntimeWarning: invalid value encountered in divide
  ax2.plot(helper.mute_energies, ufluxes_pi[t][d]/ufluxes_tot[t][d], label=r'Fraction from $K$', color=colors[0])
/tmp/ipykernel_4025577/2962969493.py:51: RuntimeWarning: invalid value encountered in divide
  ax2.plot(helper.mute_energies, ufluxes_k[t][d]/ufluxes_tot[t][d], label=r'Fraction from $\pi$', color=colors[1])
/tmp/ipykernel_4025577/2962969493.py:52: RuntimeWarning: invalid value encountered in divide
  ax2.plot(helper.mute_energies, ufluxes_pr[t][d]/ufluxes_tot[t][d], label=r'Fraction from prompt', color=colors[2])
/tmp/ipykernel_

<Figure size 2067.5x1550.62 with 0 Axes>

<Figure size 2067.5x1550.62 with 0 Axes>

<Figure size 2067.5x1550.62 with 0 Axes>

<Figure size 2067.5x1550.62 with 0 Axes>

...

<Figure size 2067.5x1550.62 with 0 Axes>

<Figure size 2067.5x1550.62 with 0 Axes>

<Figure size 2067.5x1550.62 with 0 Axes>

<Figure size 2067.5x1550.62 with 0 Axes>

<Figure size 2067.5x1550.62 with 0 Axes>

: 